# PINN on heat equation

## Stating the problem
Heat equations are Initial Boundary Value Problems (IBVPs) which means ..
#### Equation
1. $\frac{\delta u(x,y,t) }{\delta t} = c^2(\frac{\delta^2 u(x,y,t)}{\delta x^2}+\frac{\delta^2 u(x,y,t)}{\delta y^2})$ or $ u_t = c^2 (u_{xx} + u_{yy}) $


#### Boundary Conditions
2. Left side fridge
$ \frac{\delta u(0,y,t)}{\delta x} = 0 $ or $u_x(0, y, t) = 0$
3. Right side fridge
$ \frac{\delta u(length_x,y,t)}{\delta x} = 0 $ or $u_x(length_x, y, t) = 0$
4. Back side fridge
$ \frac{\delta u(x,length_y,t)}{\delta y} = 0 $ or $u_y(x, length_y, t) = 0$
5. Front side fridge
When door is closed there should be no heat transfer.

$ \frac{\delta u(x,0,t)}{\delta y} = 0 $ or $u_y(x, 0, t) = 0$

When the door is open there should be.

$ u(x,0,t) = outside\_temp$


For each of these four equations the PINN will try to satisfy them as much as possible.
This is done by minimizing the five equations:

1. $u_t - c^2 (u_{xx}+u_{yy})$
2. $u_x(0,y,t)$
3. $u_x(length_x,y,t)$
4. $u_y(x,length_y,t)$
5. $u_y(x,0,t)$ or $u(x,0,t) - outside\_temp$ conditionally on door.

Since these five should all be zero.


In [3]:
import deepxde as dde
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [27]:
c = 0.4
xL = 82
yL = 77
tL = 38266
n = 1

geom = dde.geometry.Rectangle([0, 0], [xL, yL])
timedomain = dde.geometry.TimeDomain(0, tL)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

In [28]:
def pde(x, y):
    dy_t = dde.grad.jacobian(y, x, i=0, j=0)
    dy_xx = dde.grad.hessian(y, x, i=0, j=1)
    dy_yy = dde.grad.hessian(y, x, i=0, j=2)
    return dy_t - c ** 2 * (dy_xx + dy_yy)

In [29]:
bc = dde.icbc.NeumannBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(
    geomtime,
    lambda x: 0,
    lambda _, on_initial: on_initial,
)

In [30]:
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=2540,
    num_boundary=80,
    num_initial=160,
    num_test=2540,
)

In [26]:
print(data.train_points().shape)

(2780, 3)


In [34]:
net = dde.nn.FNN([3] + [20] * 3 + [1], "tanh", "Glorot normal")

In [35]:
model = dde.Model(data, net)
model.compile("adam", lr=1e-3)

Compiling model...
'compile' took 0.003052 s



In [36]:
losshistory, train_state = model.train(iterations=10000)

Training model...

0         [9.91e-05, 6.72e-11, 3.87e-01]    [3.41e-07, 6.72e-11, 3.87e-01]    []  
1000      [4.88e-07, 9.40e-10, 1.58e-05]    [4.54e-07, 9.40e-10, 1.58e-05]    []  
2000      [3.17e-07, 3.15e-10, 5.56e-06]    [4.42e-07, 3.15e-10, 5.56e-06]    []  
3000      [2.66e-07, 1.82e-11, 2.64e-06]    [3.50e-07, 1.82e-11, 2.64e-06]    []  
4000      [2.41e-07, 9.35e-12, 1.23e-06]    [3.39e-07, 9.35e-12, 1.23e-06]    []  
5000      [1.97e-07, 6.70e-12, 6.38e-07]    [3.48e-07, 6.70e-12, 6.38e-07]    []  
6000      [1.52e-07, 7.15e-12, 3.76e-07]    [3.78e-07, 7.15e-12, 3.76e-07]    []  
7000      [1.18e-07, 7.57e-12, 2.57e-07]    [4.56e-07, 7.57e-12, 2.57e-07]    []  
8000      [1.14e-07, 1.23e-11, 2.07e-07]    [7.46e-07, 1.23e-11, 2.07e-07]    []  
9000      [9.73e-08, 1.63e-11, 1.62e-07]    [8.19e-07, 1.63e-11, 1.62e-07]    []  
10000     [8.48e-08, 1.99e-11, 1.31e-07]    [9.31e-07, 1.99e-11, 1.31e-07]    []  

Best model at step 10000:
  train loss: 2.16e-07
  test loss: 1.06e

In [12]:
model.compile("L-BFGS-B")
losshistory, train_state = model.train()

Compiling model...
'compile' took 0.005034 s

Training model...

Step      Train loss                        Test loss                         Test metric
20000     [4.59e-06, 6.21e-07, 1.02e-07]    [3.26e-06, 6.21e-07, 1.02e-07]    []  
21000     [4.36e-06, 5.13e-07, 1.48e-08]    [2.88e-06, 5.13e-07, 1.48e-08]    []  
22000     [4.36e-06, 5.12e-07, 1.47e-08]    [2.87e-06, 5.12e-07, 1.47e-08]    []  
23000     [4.36e-06, 5.12e-07, 1.45e-08]    [2.87e-06, 5.12e-07, 1.45e-08]    []  
24000     [4.35e-06, 5.12e-07, 1.44e-08]    [2.87e-06, 5.12e-07, 1.44e-08]    []  
25000     [4.35e-06, 5.11e-07, 1.43e-08]    [2.87e-06, 5.11e-07, 1.43e-08]    []  
26000     [4.35e-06, 5.11e-07, 1.42e-08]    [2.87e-06, 5.11e-07, 1.42e-08]    []  
27000     [4.35e-06, 5.10e-07, 1.41e-08]    [2.87e-06, 5.10e-07, 1.41e-08]    []  
28000     [4.34e-06, 5.10e-07, 1.39e-08]    [2.86e-06, 5.10e-07, 1.39e-08]    []  
29000     [4.34e-06, 5.10e-07, 1.38e-08]    [2.86e-06, 5.10e-07, 1.38e-08]    []  
30000     [4.34

KeyboardInterrupt: 